# Install Julia Kernel

---



In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.9.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# Methods

---



The concept of multiple dispatch is something we use frequently but never really stop to contemplate.  Julia surfaces multiple dispatch in a concrete and easy fashion compared to operator overloading or templating.

In [ ]:
f(a::Int64, b::Int64) = a + b

f(a::Float64, b::Float64) = a * b

f(a::Number, b::Number) = 2 * (a + b)

println(f(2 , 3))
println(f(2.0, 3.0))
println(f(2, 3.0))

methods(f)

In [ ]:
methods(+)

# Linear Regression - Julia


In [ ]:
import Pkg
Pkg.add("Plots")
Pkg.add("GLM")
Pkg.add("DataFrames")
using DataFrames
using BenchmarkTools
using GLM
using Plots

# perform the ordinary least squares fits
data1 = DataFrame(X1=collect(0:100), Y1=rand(0:100,101))
data2 = DataFrame(X2=collect(0:1000), Y2=rand(0:100,1001))
data3 = DataFrame(X3=collect(0:10000), Y3=rand(0:100,10001))
data4 = DataFrame(X4=collect(0:100000), Y4=rand(0:100,100001))

ols1 = @btime lm(@formula(Y1 ~ X1), data1)
ols2 = @btime lm(@formula(Y2 ~ X2), data2)
ols3 = @btime lm(@formula(Y3 ~ X3), data3)
ols4 = @btime lm(@formula(Y4 ~ X4), data4)

# plot the points
plot(X1, Y1, linewidth=2, title="X vs Y", label=["Y"], xlabel="X", ylabel="Y")

# display the coefficients
println(coef(ols1))

Yp = predict(ols1); #load predictions into Yp
Yn = Base.hcat(Y1,Yp) #concatenating Y and Yp arrays into a new one called 'Y2' so that we can plot both, below
plot(X1, Yn, linewidth=2, title="X vs Y", label=["Y", "Yp"], xlabel="X", ylabel="Y")

# Linear Regression - Python

In [ ]:

import statsmodels.api as sm
import numpy as np
import time

# defining the variables
x1 = list(range(101))
x2 = list(range(1001))
x3 = list(range(10001))
x4 = list(range(100001))
y1 = np.random.uniform(low=0, high=100, size=(101,))
y2 = np.random.uniform(low=0, high=100, size=(1001,))
y3 = np.random.uniform(low=0, high=100, size=(10001,))
y4 = np.random.uniform(low=0, high=100, size=(100001,))

# performing the regression
# and fitting the model
starttime = time.perf_counter()
result = sm.OLS(y1, x1).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

starttime = time.perf_counter()
result = sm.OLS(y2, x2).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

starttime = time.perf_counter()
result = sm.OLS(y3, x3).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

starttime = time.perf_counter()
result = sm.OLS(y4, x4).fit()
endtime = time.perf_counter()
elapsed = endtime - starttime
print(f'Time taken: {elapsed/10E-6} microseconds')

# printing the summary table
print(result.summary())
